In [1]:
import numpy as np #importing numpy
import pandas as pd #importing pandas
import PIL #importing pillow for image processing
from PIL import Image#importing image from pillow library
import skimage#importing skimage library
import os#importing os library
import time#importing time library

#Capture start time of the program
start_time = time.time()

In [2]:
from string import digits
#code to remove numbers in a string and assigning it to the variable
r_digits = str.maketrans('', '', digits)
#taking source data folder path to a varible
rawdata_path = 'D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//64//data//'
#changing current librbay to source data
os.chdir(rawdata_path)
#taking sub-directories to a list
dirs = os.listdir()
directories = []
diseases = []
#Taking directories into a list and extract disease name from folder name
for x in dirs:
    res = x.translate(r_digits).lower()
    diseases.append(res)
    temp = rawdata_path + x + '//'
    directories.append(temp)

In [3]:
for i in directories:
    os.chdir(i)
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        image = Image.open(j,mode='r')
        size = (64,64)
        image = image.resize(size)
        image = image.save(j)

In [4]:
#creating different dataframes for saving data in different vector forms
#gray data
gray_df = pd.DataFrame(columns=['mean','disease'])
#rgb data
rgb_df = pd.DataFrame(columns=['mean','disease'])
#red data
red_df = pd.DataFrame(columns=['mean','disease'])
#green data
green_df = pd.DataFrame(columns=['mean','disease'])
#blue data
blue_df = pd.DataFrame(columns=['mean','disease'])

In [5]:
#extracting data from images
x = 0
#creating enpty directories to save the vector from in to a list
gray_test = []
rgb_final = []
red_final = []
green_final = []
blue_final = []
disease = []
#reading data directory by directory
for i in directories:
    os.chdir(i)
    k = diseases[x]
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        temp_list = []
        rgb_temp = []
        red_temp = []
        green_temp = []
        blue_temp = []
        gray = []
        data = []
        red = []
        green = []
        blue = []
        #opening the image in rgb format
        image = Image.open(j,mode='r')
        data = image.convert('RGB')
        data = np.array(data)
        #extracting only red components
        red = data[:,:,0]
        #extracting only green components
        green = data[:,:,1]
        #extracting only blue components
        blue = data[:,:,2]
        data = data.ravel()
        #saving rgb vector data into list
        rgb_final.append(data)
        rgb_temp.append(np.mean(data))
        rgb_temp.append(k)
        rgb_df.loc[len(rgb_df)] = rgb_temp
        #saving red channel data into dataframe and vector data to a list.
        red = red.ravel()
        red_final.append(red)
        red_temp.append(np.mean(red))
        red_temp.append(k)
        red_df.loc[len(red_df)] = red_temp
        #saving blue channel data into dataframe and vector data to a list.
        blue = blue.ravel()
        blue_final.append(blue)
        blue_temp.append(np.mean(blue))
        blue_temp.append(k)
        blue_df.loc[len(blue_df)] = blue_temp
        #saving green channel data into dataframe and vector data to a list.
        green = green.ravel()
        green_final.append(green)
        green_temp.append(np.mean(green))
        green_temp.append(k)
        green_df.loc[len(green_df)] = green_temp
        #opening the image in gray format
        gray = image.convert('L')
        gray = np.array(gray)
        gray = gray.ravel()
        gray_test.append(gray)
        temp_list.append(np.mean(gray))
        temp_list.append(k)       
        gray_df.loc[len(gray_df)] = temp_list
    x = x+1

In [6]:
#changing path
os.chdir('D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//64//vector//')

In [7]:
#saving the lists as numpy arrays in the local system
np.save('gray_test',gray_test)
np.save('rgb_data',rgb_final)
np.save('red_data',red_final)
np.save('green_data',green_final)
np.save('blue_data',blue_final)

In [8]:
#saving the dataframe as csv files in the local system
gray_df.to_csv('gray_data.csv',index=False)
rgb_df.to_csv('rgb_data.csv',index=False)
red_df.to_csv('red_data.csv',index=False)
green_df.to_csv('green_data.csv',index=False)
blue_df.to_csv('blue_data.csv',index=False)

# KNN on rgb data

In [9]:
#reading numpy array data
rgb_test = np.load('rgb_data.npy')
#reading csv file
rgb_df = pd.read_csv('rgb_data.csv')

In [10]:
loaded = []
for i in rgb_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [11]:
#assigning features and target
features = loaded
target = rgb_df['disease']
#splitting the data to 80 percent and 20 percent
from sklearn.model_selection import train_test_split
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=24)
#scaling the data using standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [12]:
#K-Model on the data
from sklearn.neighbors import KNeighborsClassifier
kmodel = KNeighborsClassifier(n_neighbors=75)

In [13]:
#k-model fitting and accuracy
kmodel.fit(Ftrain,Ltrain)
print(kmodel.score(Ftest,Ltest))

0.5730994152046783


In [14]:
#importing classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
#printing results of the kmodel
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.33      0.86      0.48       345
                 early_blight       0.00      0.00      0.00       153
                      healthy       0.50      0.84      0.63       234
                  late_blight       0.59      0.43      0.50       309
                    leaf_mold       0.71      0.53      0.61       145
           septoria_leaf_spot       0.68      0.48      0.56       293
                  target_spot       0.42      0.45      0.43       240
          tomato_mosaic_virus       0.61      0.37      0.46        54
tomato_yellow_leaf_curl_virus       0.89      0.72      0.80       854
      two-spotted_spider_mite       0.77      0.29      0.42       280

                     accuracy                           0.57      2907
                    macro avg       0.55      0.50      0.49      2907
                 weighted avg       0.63      0.57      0.57      2907

[[2

# Logistic on rgb data

In [16]:
#importing logistic data
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
print(model.score(Ftest,Ltest))

0.7296181630546955


In [17]:
#classification report
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.70      0.80      0.74       345
                 early_blight       0.34      0.35      0.34       153
                      healthy       0.74      0.71      0.73       234
                  late_blight       0.56      0.64      0.60       309
                    leaf_mold       0.62      0.62      0.62       145
           septoria_leaf_spot       0.69      0.67      0.68       293
                  target_spot       0.69      0.64      0.66       240
          tomato_mosaic_virus       0.65      0.85      0.74        54
tomato_yellow_leaf_curl_virus       0.90      0.87      0.89       854
      two-spotted_spider_mite       0.86      0.70      0.77       280

                     accuracy                           0.73      2907
                    macro avg       0.67      0.69      0.68      2907
                 weighted avg       0.74      0.73      0.73      2907

[[2

# SVM on RGB data

In [18]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.762297901616787


In [19]:
#classification report
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.68      0.83      0.75       345
                 early_blight       0.34      0.39      0.36       153
                      healthy       0.77      0.81      0.79       234
                  late_blight       0.63      0.66      0.65       309
                    leaf_mold       0.67      0.63      0.65       145
           septoria_leaf_spot       0.71      0.66      0.68       293
                  target_spot       0.77      0.72      0.75       240
          tomato_mosaic_virus       0.76      0.83      0.80        54
tomato_yellow_leaf_curl_virus       0.94      0.89      0.91       854
      two-spotted_spider_mite       0.92      0.76      0.83       280

                     accuracy                           0.76      2907
                    macro avg       0.72      0.72      0.72      2907
                 weighted avg       0.77      0.76      0.77      2907

[[2

# Random Forest on RGB 

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,n_estimators=100, oob_score=True)

In [22]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [23]:
classifier_rf.oob_score_

0.5593599449415003

In [24]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.72      0.57      0.63       345
                 early_blight       0.00      0.00      0.00       153
                      healthy       0.55      0.91      0.68       234
                  late_blight       0.57      0.25      0.34       309
                    leaf_mold       0.74      0.34      0.46       145
           septoria_leaf_spot       0.58      0.44      0.50       293
                  target_spot       0.57      0.47      0.51       240
          tomato_mosaic_virus       0.00      0.00      0.00        54
tomato_yellow_leaf_curl_virus       0.52      0.98      0.68       854
      two-spotted_spider_mite       0.82      0.08      0.15       280

                     accuracy                           0.56      2907
                    macro avg       0.51      0.40      0.40      2907
                 weighted avg       0.57      0.56      0.50      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Bayes on RGB data

In [25]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [26]:
y_pred = gnb.predict(Ftest)

In [27]:
from sklearn import metrics
metrics.accuracy_score(Ltest, y_pred)

0.523563811489508

In [28]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.54      0.76      0.63       345
                 early_blight       0.23      0.30      0.26       153
                      healthy       0.45      0.72      0.55       234
                  late_blight       0.50      0.28      0.36       309
                    leaf_mold       0.55      0.46      0.50       145
           septoria_leaf_spot       0.44      0.46      0.45       293
                  target_spot       0.55      0.53      0.54       240
          tomato_mosaic_virus       0.20      0.72      0.31        54
tomato_yellow_leaf_curl_virus       0.89      0.52      0.66       854
      two-spotted_spider_mite       0.45      0.52      0.48       280

                     accuracy                           0.52      2907
                    macro avg       0.48      0.53      0.47      2907
                 weighted avg       0.59      0.52      0.53      2907

[[2

# Decision Tree on RGB data

In [29]:
from sklearn.tree import DecisionTreeClassifier
classifier_tree = DecisionTreeClassifier()

In [30]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.4932920536635707

In [31]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.48      0.54      0.51       345
                 early_blight       0.19      0.18      0.19       153
                      healthy       0.57      0.63      0.60       234
                  late_blight       0.43      0.46      0.44       309
                    leaf_mold       0.43      0.43      0.43       145
           septoria_leaf_spot       0.39      0.38      0.39       293
                  target_spot       0.35      0.38      0.36       240
          tomato_mosaic_virus       0.15      0.17      0.16        54
tomato_yellow_leaf_curl_virus       0.73      0.66      0.69       854
      two-spotted_spider_mite       0.39      0.36      0.37       280

                     accuracy                           0.50      2907
                    macro avg       0.41      0.42      0.41      2907
                 weighted avg       0.50      0.50      0.50      2907

[[1

# KNN on grayscale data

In [32]:
#reading numpy array data
gray_test = np.load('gray_test.npy')
#reading csv file
gray_df = pd.read_csv('gray_data.csv')

In [33]:
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [34]:
features = loaded
target = gray_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [35]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.4674922600619195

In [36]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.33      0.81      0.47       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.39      0.65      0.49       258
                  late_blight       0.34      0.31      0.33       316
                    leaf_mold       0.63      0.48      0.55       145
           septoria_leaf_spot       0.49      0.47      0.48       262
                  target_spot       0.50      0.32      0.39       229
          tomato_mosaic_virus       0.50      0.05      0.09        58
tomato_yellow_leaf_curl_virus       0.65      0.55      0.60       857
      two-spotted_spider_mite       0.77      0.27      0.40       251

                     accuracy                           0.47      2907
                    macro avg       0.46      0.39      0.38      2907
                 weighted avg       0.49      0.47      0.45      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on grayscale data

In [37]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.3804609563123495

In [38]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.53      0.54      0.54       349
                 early_blight       0.15      0.17      0.16       182
                      healthy       0.37      0.41      0.39       258
                  late_blight       0.29      0.32      0.31       316
                    leaf_mold       0.32      0.39      0.35       145
           septoria_leaf_spot       0.31      0.34      0.32       262
                  target_spot       0.27      0.28      0.28       229
          tomato_mosaic_virus       0.09      0.16      0.12        58
tomato_yellow_leaf_curl_virus       0.51      0.46      0.48       857
      two-spotted_spider_mite       0.42      0.27      0.33       251

                     accuracy                           0.38      2907
                    macro avg       0.33      0.33      0.33      2907
                 weighted avg       0.39      0.38      0.38      2907

[[1

# SVM on Grayscale data

In [39]:
clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.38321293429652564


In [40]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.40      0.56      0.47       349
                 early_blight       0.14      0.20      0.17       182
                      healthy       0.43      0.47      0.45       258
                  late_blight       0.31      0.37      0.34       316
                    leaf_mold       0.30      0.37      0.33       145
           septoria_leaf_spot       0.32      0.35      0.33       262
                  target_spot       0.26      0.32      0.29       229
          tomato_mosaic_virus       0.19      0.28      0.23        58
tomato_yellow_leaf_curl_virus       0.64      0.39      0.48       857
      two-spotted_spider_mite       0.55      0.31      0.40       251

                     accuracy                           0.38      2907
                    macro avg       0.35      0.36      0.35      2907
                 weighted avg       0.43      0.38      0.39      2907

[[1

# Random Forest on Grayscale

In [41]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [42]:
classifier_rf.oob_score_

0.45010323468685476

In [43]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.62      0.48      0.54       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.60      0.40      0.48       258
                  late_blight       0.72      0.22      0.34       316
                    leaf_mold       0.76      0.09      0.16       145
           septoria_leaf_spot       0.59      0.48      0.53       262
                  target_spot       0.69      0.16      0.26       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.40      0.92      0.56       857
      two-spotted_spider_mite       0.58      0.26      0.36       251

                     accuracy                           0.47      2907
                    macro avg       0.49      0.30      0.32      2907
                 weighted avg       0.52      0.47      0.42      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive bayes on Grayscale

In [44]:
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [45]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3780529755761954

In [46]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.43      0.74      0.54       349
                 early_blight       0.25      0.32      0.28       182
                      healthy       0.33      0.66      0.44       258
                  late_blight       0.40      0.22      0.28       316
                    leaf_mold       0.43      0.30      0.35       145
           septoria_leaf_spot       0.35      0.50      0.41       262
                  target_spot       0.53      0.38      0.44       229
          tomato_mosaic_virus       0.13      0.59      0.22        58
tomato_yellow_leaf_curl_virus       0.65      0.16      0.25       857
      two-spotted_spider_mite       0.40      0.46      0.43       251

                     accuracy                           0.38      2907
                    macro avg       0.39      0.43      0.36      2907
                 weighted avg       0.46      0.38      0.36      2907

[[2

# Decision Tree on Grayscale

In [47]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.42724458204334365

In [48]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.42      0.48      0.45       349
                 early_blight       0.15      0.12      0.13       182
                      healthy       0.56      0.56      0.56       258
                  late_blight       0.41      0.35      0.38       316
                    leaf_mold       0.26      0.30      0.28       145
           septoria_leaf_spot       0.37      0.41      0.39       262
                  target_spot       0.35      0.33      0.34       229
          tomato_mosaic_virus       0.06      0.09      0.07        58
tomato_yellow_leaf_curl_virus       0.53      0.51      0.52       857
      two-spotted_spider_mite       0.38      0.37      0.37       251

                     accuracy                           0.42      2907
                    macro avg       0.35      0.35      0.35      2907
                 weighted avg       0.42      0.42      0.42      2907

[[1

# KNN on Red channel data

In [49]:
#reading numpy array data
red_test = np.load('red_data.npy')
#reading csv file
red_df = pd.read_csv('red_data.csv')

In [50]:
loaded = []
for i in red_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = red_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [51]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.500515995872033

In [52]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.78      0.51       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.42      0.81      0.55       258
                  late_blight       0.40      0.32      0.36       316
                    leaf_mold       0.62      0.48      0.54       145
           septoria_leaf_spot       0.54      0.45      0.49       262
                  target_spot       0.48      0.51      0.49       229
          tomato_mosaic_virus       0.35      0.10      0.16        58
tomato_yellow_leaf_curl_virus       0.64      0.55      0.59       857
      two-spotted_spider_mite       0.75      0.37      0.50       251

                     accuracy                           0.50      2907
                    macro avg       0.46      0.44      0.42      2907
                 weighted avg       0.51      0.50      0.48      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on Red channel data

In [53]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.4282765737874097

In [54]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.56      0.59      0.57       349
                 early_blight       0.14      0.16      0.15       182
                      healthy       0.47      0.52      0.49       258
                  late_blight       0.34      0.35      0.35       316
                    leaf_mold       0.34      0.43      0.38       145
           septoria_leaf_spot       0.32      0.37      0.34       262
                  target_spot       0.40      0.38      0.39       229
          tomato_mosaic_virus       0.16      0.22      0.19        58
tomato_yellow_leaf_curl_virus       0.55      0.49      0.52       857
      two-spotted_spider_mite       0.50      0.35      0.41       251

                     accuracy                           0.43      2907
                    macro avg       0.38      0.39      0.38      2907
                 weighted avg       0.44      0.43      0.43      2907

[[2

# SVM on Red channel data

In [55]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.42655658754729964


In [56]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.62      0.49       349
                 early_blight       0.15      0.21      0.17       182
                      healthy       0.50      0.54      0.52       258
                  late_blight       0.34      0.37      0.35       316
                    leaf_mold       0.32      0.37      0.34       145
           septoria_leaf_spot       0.34      0.38      0.36       262
                  target_spot       0.36      0.35      0.36       229
          tomato_mosaic_virus       0.23      0.38      0.29        58
tomato_yellow_leaf_curl_virus       0.68      0.42      0.52       857
      two-spotted_spider_mite       0.63      0.46      0.53       251

                     accuracy                           0.43      2907
                    macro avg       0.40      0.41      0.39      2907
                 weighted avg       0.47      0.43      0.44      2907

[[2

# Random forest on Red channel data

In [57]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.4869236063317275

In [58]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.64      0.49      0.56       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.70      0.43      0.54       258
                  late_blight       0.61      0.17      0.27       316
                    leaf_mold       0.81      0.12      0.20       145
           septoria_leaf_spot       0.53      0.48      0.50       262
                  target_spot       0.73      0.23      0.35       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.42      0.91      0.58       857
      two-spotted_spider_mite       0.61      0.49      0.54       251

                     accuracy                           0.50      2907
                    macro avg       0.50      0.33      0.35      2907
                 weighted avg       0.53      0.50      0.44      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Bayes on Red channel data

In [59]:
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [60]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3935328517371861

In [61]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.46      0.72      0.56       349
                 early_blight       0.22      0.26      0.24       182
                      healthy       0.36      0.72      0.47       258
                  late_blight       0.44      0.24      0.31       316
                    leaf_mold       0.37      0.32      0.34       145
           septoria_leaf_spot       0.36      0.46      0.40       262
                  target_spot       0.55      0.47      0.51       229
          tomato_mosaic_virus       0.15      0.69      0.25        58
tomato_yellow_leaf_curl_virus       0.63      0.14      0.23       857
      two-spotted_spider_mite       0.43      0.59      0.50       251

                     accuracy                           0.39      2907
                    macro avg       0.40      0.46      0.38      2907
                 weighted avg       0.47      0.39      0.37      2907

[[2

# Decision Tree on Red channel

In [62]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.4444444444444444

In [63]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.47      0.51      0.49       349
                 early_blight       0.16      0.15      0.15       182
                      healthy       0.61      0.64      0.62       258
                  late_blight       0.46      0.40      0.43       316
                    leaf_mold       0.27      0.28      0.27       145
           septoria_leaf_spot       0.35      0.40      0.37       262
                  target_spot       0.30      0.28      0.29       229
          tomato_mosaic_virus       0.11      0.17      0.14        58
tomato_yellow_leaf_curl_virus       0.59      0.55      0.57       857
      two-spotted_spider_mite       0.41      0.44      0.42       251

                     accuracy                           0.45      2907
                    macro avg       0.37      0.38      0.38      2907
                 weighted avg       0.45      0.45      0.45      2907

[[1

# KNN on green channel data

In [64]:
#reading numpy array data
green_test = np.load('green_data.npy')
#reading csv file
green_df = pd.read_csv('green_data.csv')
loaded = []
for i in green_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = green_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.4582043343653251

In [65]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.31      0.80      0.44       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.38      0.60      0.46       258
                  late_blight       0.31      0.31      0.31       316
                    leaf_mold       0.51      0.50      0.50       145
           septoria_leaf_spot       0.52      0.47      0.50       262
                  target_spot       0.49      0.22      0.31       229
          tomato_mosaic_virus       0.40      0.07      0.12        58
tomato_yellow_leaf_curl_virus       0.70      0.57      0.63       857
      two-spotted_spider_mite       0.68      0.24      0.36       251

                     accuracy                           0.46      2907
                    macro avg       0.43      0.38      0.36      2907
                 weighted avg       0.49      0.46      0.44      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on Green channel data

In [66]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.3935328517371861

In [67]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.52      0.54      0.53       349
                 early_blight       0.18      0.19      0.18       182
                      healthy       0.43      0.46      0.45       258
                  late_blight       0.31      0.36      0.33       316
                    leaf_mold       0.31      0.35      0.33       145
           septoria_leaf_spot       0.30      0.33      0.31       262
                  target_spot       0.29      0.32      0.30       229
          tomato_mosaic_virus       0.11      0.17      0.13        58
tomato_yellow_leaf_curl_virus       0.53      0.47      0.50       857
      two-spotted_spider_mite       0.40      0.27      0.32       251

                     accuracy                           0.39      2907
                    macro avg       0.34      0.35      0.34      2907
                 weighted avg       0.41      0.39      0.40      2907

[[1

# SVM on green channel data

In [68]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.3928448572411421


In [69]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.54      0.45       349
                 early_blight       0.19      0.29      0.23       182
                      healthy       0.44      0.48      0.46       258
                  late_blight       0.31      0.37      0.34       316
                    leaf_mold       0.26      0.32      0.29       145
           septoria_leaf_spot       0.30      0.32      0.31       262
                  target_spot       0.27      0.32      0.29       229
          tomato_mosaic_virus       0.23      0.26      0.25        58
tomato_yellow_leaf_curl_virus       0.67      0.42      0.52       857
      two-spotted_spider_mite       0.55      0.32      0.40       251

                     accuracy                           0.39      2907
                    macro avg       0.36      0.36      0.35      2907
                 weighted avg       0.44      0.39      0.40      2907

[[1

# Random Forest on green channel data

In [70]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.4338437715072264

In [71]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.57      0.50      0.53       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.57      0.44      0.50       258
                  late_blight       0.65      0.25      0.36       316
                    leaf_mold       0.00      0.00      0.00       145
           septoria_leaf_spot       0.61      0.45      0.52       262
                  target_spot       0.00      0.00      0.00       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.38      0.92      0.54       857
      two-spotted_spider_mite       0.50      0.01      0.02       251

                     accuracy                           0.44      2907
                    macro avg       0.33      0.26      0.25      2907
                 weighted avg       0.40      0.44      0.35      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive bayes on Green channel data

In [72]:
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.36807705538355695

In [73]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.71      0.52       349
                 early_blight       0.26      0.35      0.30       182
                      healthy       0.36      0.63      0.45       258
                  late_blight       0.40      0.22      0.29       316
                    leaf_mold       0.35      0.28      0.31       145
           septoria_leaf_spot       0.31      0.50      0.38       262
                  target_spot       0.52      0.37      0.43       229
          tomato_mosaic_virus       0.14      0.59      0.22        58
tomato_yellow_leaf_curl_virus       0.65      0.16      0.25       857
      two-spotted_spider_mite       0.39      0.40      0.39       251

                     accuracy                           0.37      2907
                    macro avg       0.38      0.42      0.35      2907
                 weighted avg       0.45      0.37      0.35      2907

[[2

# Decision Tree on Green channel

In [74]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.4176126590987272

In [75]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.44      0.46      0.45       349
                 early_blight       0.17      0.15      0.16       182
                      healthy       0.56      0.53      0.54       258
                  late_blight       0.40      0.34      0.37       316
                    leaf_mold       0.22      0.27      0.24       145
           septoria_leaf_spot       0.35      0.44      0.39       262
                  target_spot       0.29      0.26      0.27       229
          tomato_mosaic_virus       0.14      0.14      0.14        58
tomato_yellow_leaf_curl_virus       0.58      0.54      0.56       857
      two-spotted_spider_mite       0.35      0.42      0.38       251

                     accuracy                           0.42      2907
                    macro avg       0.35      0.35      0.35      2907
                 weighted avg       0.42      0.42      0.42      2907

[[1

# KNN on blue channel data

In [76]:
#reading numpy array data
blue_test = np.load('blue_data.npy')
#reading csv file
blue_df = pd.read_csv('blue_data.csv')
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = blue_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.4674922600619195

In [77]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.33      0.81      0.47       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.39      0.65      0.49       258
                  late_blight       0.34      0.31      0.33       316
                    leaf_mold       0.63      0.48      0.55       145
           septoria_leaf_spot       0.49      0.47      0.48       262
                  target_spot       0.50      0.32      0.39       229
          tomato_mosaic_virus       0.50      0.05      0.09        58
tomato_yellow_leaf_curl_virus       0.65      0.55      0.60       857
      two-spotted_spider_mite       0.77      0.27      0.40       251

                     accuracy                           0.47      2907
                    macro avg       0.46      0.39      0.38      2907
                 weighted avg       0.49      0.47      0.45      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# logistic on blue channel data

In [78]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.3804609563123495

In [79]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.53      0.54      0.54       349
                 early_blight       0.15      0.17      0.16       182
                      healthy       0.37      0.41      0.39       258
                  late_blight       0.29      0.32      0.31       316
                    leaf_mold       0.32      0.39      0.35       145
           septoria_leaf_spot       0.31      0.34      0.32       262
                  target_spot       0.27      0.28      0.28       229
          tomato_mosaic_virus       0.09      0.16      0.12        58
tomato_yellow_leaf_curl_virus       0.51      0.46      0.48       857
      two-spotted_spider_mite       0.42      0.27      0.33       251

                     accuracy                           0.38      2907
                    macro avg       0.33      0.33      0.33      2907
                 weighted avg       0.39      0.38      0.38      2907

[[1

# SVM on blue channel data

In [80]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.38321293429652564


In [81]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.40      0.56      0.47       349
                 early_blight       0.14      0.20      0.17       182
                      healthy       0.43      0.47      0.45       258
                  late_blight       0.31      0.37      0.34       316
                    leaf_mold       0.30      0.37      0.33       145
           septoria_leaf_spot       0.32      0.35      0.33       262
                  target_spot       0.26      0.32      0.29       229
          tomato_mosaic_virus       0.19      0.28      0.23        58
tomato_yellow_leaf_curl_virus       0.64      0.39      0.48       857
      two-spotted_spider_mite       0.55      0.31      0.40       251

                     accuracy                           0.38      2907
                    macro avg       0.35      0.36      0.35      2907
                 weighted avg       0.43      0.38      0.39      2907

[[1

# Random Forest on blue channel data

In [82]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.45010323468685476

In [83]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.62      0.48      0.54       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.60      0.40      0.48       258
                  late_blight       0.72      0.22      0.34       316
                    leaf_mold       0.76      0.09      0.16       145
           septoria_leaf_spot       0.59      0.48      0.53       262
                  target_spot       0.69      0.16      0.26       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.40      0.92      0.56       857
      two-spotted_spider_mite       0.58      0.26      0.36       251

                     accuracy                           0.47      2907
                    macro avg       0.49      0.30      0.32      2907
                 weighted avg       0.52      0.47      0.42      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive bayes on blue channel data

In [84]:
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3780529755761954

In [85]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.43      0.74      0.54       349
                 early_blight       0.25      0.32      0.28       182
                      healthy       0.33      0.66      0.44       258
                  late_blight       0.40      0.22      0.28       316
                    leaf_mold       0.43      0.30      0.35       145
           septoria_leaf_spot       0.35      0.50      0.41       262
                  target_spot       0.53      0.38      0.44       229
          tomato_mosaic_virus       0.13      0.59      0.22        58
tomato_yellow_leaf_curl_virus       0.65      0.16      0.25       857
      two-spotted_spider_mite       0.40      0.46      0.43       251

                     accuracy                           0.38      2907
                    macro avg       0.39      0.43      0.36      2907
                 weighted avg       0.46      0.38      0.36      2907

[[2

# Decision Tree on Blue channel

In [86]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.42208462332301344

In [87]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.43      0.46      0.44       349
                 early_blight       0.17      0.13      0.14       182
                      healthy       0.57      0.60      0.58       258
                  late_blight       0.41      0.37      0.39       316
                    leaf_mold       0.26      0.30      0.28       145
           septoria_leaf_spot       0.37      0.42      0.39       262
                  target_spot       0.34      0.32      0.33       229
          tomato_mosaic_virus       0.12      0.16      0.13        58
tomato_yellow_leaf_curl_virus       0.54      0.52      0.53       857
      two-spotted_spider_mite       0.37      0.37      0.37       251

                     accuracy                           0.42      2907
                    macro avg       0.36      0.36      0.36      2907
                 weighted avg       0.42      0.42      0.42      2907

[[1